In [61]:
import pandas as pd
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import numpy as np
# custom-made stats functions in StatsFunctions module
import StatsFunctions

sns.set()

In [62]:
test_df = pd.read_csv(r'Data\test.csv')
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [63]:
test_df.isnull().sum()[test_df.isnull().sum() != 0]

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64

In [64]:
test_df.drop(['Alley'],axis=1,inplace=True)
test_df.drop(['GarageYrBlt'],axis=1,inplace=True)
test_df.drop(['PoolQC','Fence','MiscFeature','MiscVal'],axis=1,inplace=True)

In [65]:
test_fill_na = test_df[['LotFrontage','MasVnrType','MasVnrArea','BsmtQual','BsmtCond','BsmtExposure',
'BsmtFinType1','BsmtFinType2','Electrical','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond']]
test_fill_na_cat = [cat for cat in test_fill_na.columns if test_fill_na[cat].dtype=='object']
test_fill_na_cat = test_fill_na[test_fill_na_cat]
test_fill_na_cat.columns

Index(['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond'],
      dtype='object')

In [66]:
test_fill_na_num = [num for num in test_fill_na.columns if test_fill_na[num].dtype=='float64']
test_fill_na_num = test_fill_na[test_fill_na_num]
test_fill_na_num.columns

Index(['LotFrontage', 'MasVnrArea'], dtype='object')

In [67]:
test_df['MasVnrType'] = test_df['MasVnrType'].fillna('None')
test_df['BsmtQual'] = test_df['BsmtQual'].fillna('None')
test_df['BsmtCond'] = test_df['BsmtCond'].fillna('None')
test_df['BsmtExposure'] = test_df['BsmtExposure'].fillna('None')

test_df['BsmtFinType1'] = test_df['BsmtFinType1'].fillna('None')
test_df['BsmtFinType2'] = test_df['BsmtFinType2'].fillna('None')

test_df['Electrical'] = test_df['Electrical'].fillna('None')
test_df['FireplaceQu'] = test_df['FireplaceQu'].fillna('None')

test_df['GarageType'] = test_df['GarageType'].fillna('None')
test_df['GarageFinish'] = test_df['GarageFinish'].fillna('None')
test_df['GarageQual'] = test_df['GarageQual'].fillna('None')
test_df['GarageCond'] = test_df['GarageCond'].fillna('None')

In [68]:
test_df["LotFrontage"] = test_df.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [69]:
test_df['MasVnrArea'] = test_df['MasVnrArea'].fillna(0)

In [70]:
test_df['MSZoning'] = test_df['MSZoning'].fillna(test_df['MSZoning'].mode()[0])

In [71]:
#test_df.drop('Utilities',axis=1,inplace=True)

In [72]:
mode_col = ['Electrical','KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']
for col in mode_col:
    test_df[col] = test_df[col].fillna(test_df[col].mode()[0])

In [73]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    test_df[col] = test_df[col].fillna(0)

In [74]:
for col in ('GarageCars','GarageArea'):
    test_df[col] = test_df[col].fillna(0)

In [75]:
test_df['Functional'] = test_df['Functional'].fillna('Typ')

In [76]:
test_df.isnull().sum()[test_df.isnull().sum() != 0]

Utilities    2
dtype: int64

In [77]:
test_df['OverallCond'] = test_df['OverallCond'].astype(str)
test_df['OverallQual'] = test_df['OverallQual'].astype(str)

In [78]:
test_df['TotalSF'] = test_df['TotalBsmtSF'] + test_df['1stFlrSF'] + test_df['2ndFlrSF']

In [79]:
test_df['MSSubClass'] = test_df['MSSubClass'].apply(str)
test_df['OverallCond'] = test_df['OverallCond'].astype(str)
test_df['YrSold'] = test_df['YrSold'].astype(str)
test_df['MoSold'] = test_df['MoSold'].astype(str)

In [80]:
test_df.to_csv('Data\\test_preprocessed.csv', index=False)